# Content Based Filtering

overview, cast, crew, keyword 등등에 해당하는 영화의 contents 들을 이용하여 영화간의 similarity를 찾아내는 기법.
영화간의 유사성을 기준으로 하여 추천하게 된다.

***데이터 전처리***

사용한 데이터 : tmdb_5000_credits.csv / tmdb_5000_movies.csv


credits : movie_id, cast, crew



movies : budget, genre, homepage, id, keywords,
overview , etc,,

In [ ]:
import pandas as pd
import numpy as np
df1 = pd.read_csv('/content/drive/MyDrive/tmdb_5000_credits.csv')
df2 = pd.read_csv('/content/drive/MyDrive/tmdb_5000_movies.csv')


In [ ]:
df1

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
...,...,...,...,...
4798,9367,El Mariachi,"[{""cast_id"": 1, ""character"": ""El Mariachi"", ""c...","[{""credit_id"": ""52fe44eec3a36847f80b280b"", ""de..."
4799,72766,Newlyweds,"[{""cast_id"": 1, ""character"": ""Buzzy"", ""credit_...","[{""credit_id"": ""52fe487dc3a368484e0fb013"", ""de..."
4800,231617,"Signed, Sealed, Delivered","[{""cast_id"": 8, ""character"": ""Oliver O\u2019To...","[{""credit_id"": ""52fe4df3c3a36847f8275ecf"", ""de..."
4801,126186,Shanghai Calling,"[{""cast_id"": 3, ""character"": ""Sam"", ""credit_id...","[{""credit_id"": ""52fe4ad9c3a368484e16a36b"", ""de..."


In [ ]:
df2

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [ ]:
df1.columns = ['id','title','cast','crew'] # column name을 맞추기 위해서 movie_id 를 id로 변환
df2 = df2.merge(df1,on='id') #'id'를 기준으로 두 데이터 merge
df2.head(5)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title_x,vote_average,vote_count,title_y,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


# **1 . Plot description based Recommender**

:다양한 영화 관련 정보와 콘텐츠들 중 영화 내용을 기준으로,
영화간의 similarity score를 계산하여 recommend하는 방법

In [ ]:
# overview : 영화에 대한 간단한 설명사항을 담고 있는 변수에 해당함
df2['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

***1. 아이템 프로필 생성***

✓ TF-IDF 방법
TF-IDF로 만들어진 행렬을 토대로 클러스터링 해 대상 아이템과
같은 군집 속에 있는 아이템들을 추천해준다.
즉, 아이템 간의 연관성을 분석해 연관성이 높은 다른 아이템을 찾아서 보여준다


TF*IDF
(Term Frequency-Inverse Document Frequency)

: output > each column represents a word in the overview vocabulary, each row represents a movie

In [ ]:
#Import TfIdfVectorizer from scikit-learn / 내장되어 있는 함수 TfidVectorizer 이용하기 
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a' / 불용어 제거
tfidf = TfidfVectorizer(stop_words ='english')

#Replace NaN with an empty string / 결측치 정리
df2['overview'] =df2['overview'].fillna('')

#Construct the required TF-IDF matrix by fitting and transforming the data / tfidf 함수를 이용해서 transform
tfidf_matrix = tfidf.fit_transform(df2['overview'])
tfidf_matrix.shape

(4803, 20978)

4803개의 movie에서 20978개의 different words가 있음을 확인할 수 있다.

**2. 코사인 유사도 행렬 구하기** 

linear_kernel : cosine 유사도를 구하는 함수

In [ ]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

**3. 사용자 프로필 행렬**

In [ ]:
#Construct a reverse map of indices and movie titles / 중복값 제거, title만 저장
indices = pd.Series(df2.index, index=df2['title_x']).drop_duplicates() 

**4. 콘텐츠 기반 필터링 , 추천 시스템**

In [ ]:
# movie title과 앞에서 생성한 코사인 유사도 행렬을 이용해서 10개의 비슷한 영화를 추천하는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title 
    idx = indices[title]

    # 대입받은 영화와 모든 다른 영화 간의 pairwise 유사도 점수 도출
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores / 유사도 점수를 내림차순으로 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies / 유사도 점수가 높은 10개의 movie의 유사도를 저장
    sim_scores = sim_scores[1:11]

    # Get the movie indices / 그 10개 영화 정보를 저장
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies / 10개 영화의 title 도출
    return df2['title_x'].iloc[movie_indices]

In [ ]:
get_recommendations('The Dark Knight Rises')

65                              The Dark Knight
299                              Batman Forever
428                              Batman Returns
1359                                     Batman
3854    Batman: The Dark Knight Returns, Part 2
119                               Batman Begins
2507                                  Slow Burn
9            Batman v Superman: Dawn of Justice
1181                                        JFK
210                              Batman & Robin
Name: title_x, dtype: object

In [ ]:
get_recommendations('The Avengers')

7               Avengers: Age of Ultron
3144                            Plastic
1715                            Timecop
4124                 This Thing of Ours
3311              Thank You for Smoking
3033                      The Corruptor
588     Wall Street: Money Never Sleeps
2136         Team America: World Police
1468                       The Fountain
1286                        Snowpiercer
Name: title_x, dtype: object

# **Credits,Genres and Ketwords Based Recommender**

metadata를 사용할 경우 recommender의 quality가 increase하게 된다.
3 top actors, the director, related genres, movie plot keywords를 사용하여 
recommender을 bulilding 해볼 예정 !

In [ ]:
# stringfied lists 형태를 usable structure로 바꿔서 사용을 용이하게 함 : literal_eval 활용
from ast import literal_eval #list형태 -> 본래의 형태

features = ['cast','crew','keywords','genres']
for feature in features:
  df2[feature] = df2[feature].apply(literal_eval)

In [ ]:
# crew에서 director name을 추출하는 function , 직업이 director가 아닌 경우에는 nan 도출
def get_director(x):
  for i in x:
    if i['job'] == 'Director' :
      return i['name']
  return np.nan

In [ ]:
# top 3 elements or entire list를 추출하는 function / isinstance : 변수의 class를 검사함 , 주연배우 3명
def get_list(x):
  if isinstance(x,list):
    names = [i['name'] for i in x]
    # list의 길이가 3 이상이면 맨 앞부터 3 elements만 그 미만이면 전부 , empty list dㅣ면
    # missing data 형태로
    if len(names) > 3:
      names = names[:3]
    return names
  return []



In [ ]:
# 새로운 변수 지정, director /  그에 맞는 새로운 features 지정
df2['director'] = df2['crew'].apply(get_director)
features = ['cast','keywords','genres']
for feature in features:
  df2[feature] =df2[feature].apply(get_list)

In [ ]:
df2[['title_x','cast','director','keywords','genres']].head(3)

,title_x,cast,director,keywords,genres
0,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",James Cameron,"[culture clash, future, space war]","[Action, Adventure, Fantasy]"
1,Pirates of the Caribbean: At World's End,"[Johnny Depp, Orlando Bloom, Keira Knightley]",Gore Verbinski,"[ocean, drug abuse, exotic island]","[Adventure, Fantasy, Action]"
2,Spectre,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",Sam Mendes,"[spy, based on novel, secret agent]","[Action, Adventure, Crime]"


names와 keywords를 모두 lowercase로 바꾸고, 글자사이의 spaces를 없애는 작업, 이 과정을 거쳐야 
vectorizer가 정확하게 문자들을 구분할 수 있다.

In [ ]:
# function to convert all string to lower case and strip names of spaces
def clean_data(x):
  if isinstance(x,list):
    return[str.lower(i.replace(" ","")) for i in x]
  else:
    if isinstance(x,str):
      return str.lower(x.replace(" ",""))
    else:
      return ''
# director가 있는지 확인하고, 없다면 empty string으로 지정하는 과정 

In [ ]:
features = ['cast','keywords','director','genres']
for feature in features :
  df2[feature] = df2[feature].apply(clean_data)
  

step 1 > "metadata soup" 만들기

metadata soup : 우리가 vectorizer 할 때 필요한 모든 metadata들을 포함하고 있는 string

In [ ]:
def create_soup(x):
  return ' '.join(x['keywords'])+ ' '+ ' '.join(x['cast']) +' ' + x['director'] + ' '+ ' '.join(x['genres'])

df2['soup'] =df2.apply(create_soup,axis=1)

step 2 > CountVectorizer() 활용하여 recommender 생성

:countervectorizer을 이용하는 이유 : TF-IDF는 down-weight해버리는 요소가 있기 때문에, 이 경우 많이 등장한 actor/diirector 들이 down-weight 될 수 있으므로 !

In [ ]:
#Countvectorizer불러오기 + count matrix 생성하기
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])

In [ ]:
# count_matrix를 기준으로 cosine similarity matrix를 계산하기
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix,count_matrix)

In [ ]:
#Reset index of out main Dataframe and construct reverse mapping as before
indices = pd.Series(df2.index, index=df2['title_x'])

get_recommendations() 함수를 활용하기

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

65               The Dark Knight
119                Batman Begins
4638    Amidst the Devil's Wings
1196                The Prestige
3073           Romeo Is Bleeding
3326              Black November
1503                      Takers
1986                      Faster
303                     Catwoman
747               Gangster Squad
Name: title_x, dtype: object

In [ ]:
get_recommendations('The Godfather', cosine_sim2)

867      The Godfather: Part III
2731      The Godfather: Part II
4638    Amidst the Devil's Wings
2649           The Son of No One
1525              Apocalypse Now
1018             The Cotton Club
1170     The Talented Mr. Ripley
1209               The Rainmaker
1394               Donnie Brasco
1850                    Scarface
Name: title_x, dtype: object

추가적으로 production_company도 고려해볼 수 있고, 특정 변수의 weight를 증가시키고자 할때에는, adding feature multiple times in the soup 을 통해서 weight를 더 증가시킬 수 있다.